<a href="https://colab.research.google.com/github/IrfanKhalid/BigData-Spark/blob/master/GCP_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.cloud import storage
from io import BytesIO
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
sc = SparkContext()
sqlContext = SQLContext(sc)



In [0]:
!pip install pyspark

    100% |████████████████████████████████| 215.7MB 110kB/s 
    100% |████████████████████████████████| 204kB 34.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/47/9b/57/7984bf19763749a13eece44c3174adb6ae4bc95b920375ff50
Successfully built pyspark


In [0]:
from google.colab import files
uploaded=files.upload()

Saving My First Project-0b44d8f4fdb0.json to My First Project-0b44d8f4fdb0.json


In [0]:
#client = storage.Client()
client = storage.Client.from_service_account_json('My First Project-0b44d8f4fdb0.json')

In [0]:
bucket_name = "criminaldata"

In [0]:
bucket = client.get_bucket(bucket_name)

blob = bucket.get_blob('test.csv')

In [0]:
content = blob.download_as_string()

In [0]:
print(content)

b'ID,Case_Number,IUCR,Primary_Type,Description,,,,,,,,,,,,,,,\r\n5988306,HN776442,460,BATTERY,SIMPLE,,,,,,,,,,,,,,,\r\n5988308,HN784135,454,BATTERY,AGG PO HANDS NO/MIN INJURY,,,,,,,,,,,,,,,\r\n5988309,HN783559,610,BURGLARY,FORCIBLE ENTRY,,,,,,,,,,,,,,,\r\n5988310,HN783774,610,BURGLARY,FORCIBLE ENTRY,,,,,,,,,,,,,,,\r\n5988311,HN783646,486,BATTERY,DOMESTIC BATTERY SIMPLE,,,,,,,,,,,,,,,\r\n5988312,HN783708,810,THEFT,OVER $500,,,,,,,,,,,,,,,\r\n5988314,HN782927,560,ASSAULT,SIMPLE,,,,,,,,,,,,,,,\r\n5988315,HN783827,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,,,,,,,,,,,,,,,\r\n5988317,HN783615,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,,,,,,,,,,,,,,,\r\n5988318,HN783390,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,,,,,,,,,,,,,,,\r\n5988320,HN783115,2820,OTHER OFFENSE,TELEPHONE THREAT,,,,,,,,,,,,,,,\r\n5988321,HN780817,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,,,,,,,,,,,,,,,\r\n5988322,HN783208,1310,CRIMINAL DAMAGE,TO PROPERTY,,,,,,,,,,,,,,,\r\n5988327,HN783273,560,ASSAULT,SIMPLE,,,,,,,,,,,,,

In [0]:
data = sqlContext.createDataFrame([BytesIO(content)]) 

In [0]:
data = data.filter(train.Year == 2007)
data = data.select('Primary Type','IUCR')

In [0]:
#!/usr/bin/python

# Author: Gary A. Stafford
# License: MIT

from pyspark.sql import SparkSession


def main():
    spark = SparkSession \
        .builder \
        .master("yarn") \
        .appName('dataproc-python-demo') \
        .getOrCreate()

    # Defaults to INFO
    sc = spark.sparkContext
    sc.setLogLevel("WARN")

    # Loads CSV file from Google Storage Bucket
    df_loans = spark \
        .read \
        .format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("gs://criminaldata/crimes.csv")

    # Creates temporary view using DataFrame
    df_loans.withColumnRenamed("ID", "ID") \
        .withColumnRenamed("Primary_Type", "Primary_Type") \
        .withColumnRenamed("Date", "Date") \        
        .createOrReplaceTempView("crimes")
		
	df_loans.withColumnRenamed("ID", "ID") \
        .withColumnRenamed("Primary_Type", "Primary_Type") \
        .withColumnRenamed("Date", "Date") \
        .createOrReplaceTempView("loans")

    # Performs basic analysis of dataset
    df_disbursement = spark.sql("""
            SELECT Primary_Type As Crime, count(Primary_Type)			
            FROM crimes
            WHERE  Date like '%2007%'
			GROUP BY Primary_Type
    """).cache()

    # Saves results to single CSV file in Google Storage Bucket
    df_disbursement.repartition(1) \
        .write \
        .mode("overwrite") \
        .format("csv") \
        .option("header", "true") \
        .save("gs://criminaldata/Result")

    spark.stop()


if __name__ == "__main__":
    main()
